Python 中，数据的属性和处理数据的方法统称属性（attribute）。其实，方法只是可调用的属性。除了这二者之外，我们还可以创建特性（property），在不改变类接口的前提下，使用存取方法（即读值方法和设值方法）修改数据属性。

除了特性，Python 还提供了丰富的 API，用于控制属性的访问权限，以及实现动态属性。使用点号访问属性时，Python 会调用特殊的方法（如 `__getattr__` 和 `__setattr__`）计算属性。用户自定义的类可以通过 `__getattr__` 方法实现 “虚拟属性”，当访问不存在的属性时，即时计算属性值

动态创建属性是一种元变成，框架作者经常这么做，在 Python 中，这种技术很简单，任何人都可以使用，甚至在日常数据转换任务都能用到

## 使用动态属性转换数据

我们编写个脚本下载 OSCON 数据源，这是一份 JSON 数据，我们后面来解析它：

In [3]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = '/home/kaka/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local: # with 语句使用两个上下文管理器，用来读取和保存远程文件
            local.write(remote.read())
    with open(JSON) as fp: # 解析 JSON 文件，返回 Python 原生对象，这里数据有下面几种类型：dict, list, str, int
        return json.load(fp)
load()

{'Schedule': {'conferences': [{'serial': 115}],
  'events': [{'categories': ['Emerging Languages'],
    'description': 'The web development platform is massive. With tons of libraries, frameworks and concepts out there, it might be daunting for the &#39;legacy&#39; developer to jump into it.\r\n\r\nIn this presentation we will introduce Google Dart &amp; Polymer. Two hot technologies that work in harmony to create powerful web applications using concepts familiar to OOP developers.',
    'event_type': '40-minute conference session',
    'name': 'Migrating to the Web Using Dart and Polymer - A Guide for Legacy OOP Developers',
    'serial': 33451,
    'speakers': [149868],
    'time_start': '2014-07-23 17:00:00',
    'time_stop': '2014-07-23 17:40:00',
    'venue_serial': 1458,
    'website_url': 'https://conferences.oreilly.com/oscon/oscon2014/public/schedule/detail/33451'},
   {'categories': ['PHP'],
    'description': 'Refactoring code (altering code to make it cleaner, simpler, and 

In [4]:
feed = load()
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [5]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key)) # 显示各个集合中记录数量

  1 conferences
494 events
357 speakers
 53 venues


In [6]:
feed['Schedule']['speakers'][-1]['name'] # 深入潜逃的字典和列表，获取最后一个演讲者名字

'Carina C. Zona'

In [7]:
feed['Schedule']['speakers'][-1]['serial'] # 演讲者编号

141590

In [8]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [9]:
feed['Schedule']['events'][40]['speakers'] # 每个事件都有一个 'speakers' 字段，列出 0 个或多个演讲者编号

[3471, 5199]

## 使用动态属性访问 JSON 类数据

上面这种 feed['Schedule']['events'][40]['name']  语法太长，在 JavaScript 中可以使用 feed.Schedule.events[40].name 获取那个值，在 Python 中可以实现一个近似字典的类（网上有很多）达到同样效果，这里我们自己实现一个 FrozenJSON 类，比大多数实现都简单，因为只支持读取，即只能访问数据，不过，这个类能递归，自动处理嵌套的映射和列表

In [18]:
from collections import abc

class FrozenJSON:
    
    def __init__(self, mapping):
        # 这么做有两个目的，首先确保传入的是字典，其次为了创建一个副本
        self.__data = dict(mapping)
    
    def __getattr__(self, name): # 仅当没有 name 属性时才调用此方法(通常在属性查找找不到时候调用)
        if hasattr(self.__data, name):
            return getattr(self.__data, name) # 如果 name 是 `__data` 的属性，返回那个属性，例如字典自带的 keys 属性
        else:
            # 否则，从 self.__data 中获取 name 键对应的元素，返回调用 FrozenJSON.build() 方法得到的结果
            return FrozenJSON.build(self.__data[name]) 
    
    @classmethod 
    def build(cls, obj): # 这是一个备选构造方法，classmethod 装饰器经常这么用
        if isinstance(obj, abc.Mapping): # 如果 obj 是映射，那么构造一个 FrozenJSON 对象
            return cls(obj)
        # 如果是 MutableSequence，必然是列表。因此我们把 obj 中的每个元素递归传给 .build() 方法，构建一个列表
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            # 既不是字典又不是列表，原封不动返回
            return obj

FrozenJSON 中，尝试获取其它属性会出发解释器调用 `__getattr__` 方法，这个方法首先查看 `self.__data` 有没有指定属性名（而不是键），这样 FrozenJSON 实例便可以处理字典的所有方法，例如把 items 方法委托给 `self.__data.items()` 方法。如果 `self.__data` 没有指定名称属性，那么 `__getattr__` 方法以那个名称为键，从 `self.__data` 中获取一个元素，传给 FrozenJSON.build 方法。这样就能深入 JSON 数据的嵌套结构，使用类方法 build 把每一层嵌套转成一个 FrozenJSON 实例，我们没有缓存或转换原始数据，在迭代数据过程中，嵌套的数据结构不断被转成 FrozenJSON 对象，这么做没问题，因为我们的数据量不大

In [17]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

357

In [19]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [21]:
for key, value in sorted(feed.Schedule.items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [22]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [24]:
talk = feed.Schedule.events[40]
type(talk)

__main__.FrozenJSON

In [25]:
talk.name

'There *Will* Be Bugs'

In [26]:
talk.speakers

[3471, 5199]

In [27]:
talk.flavor # 读取不存在的实例抛出 KeyError 异常，而不是通常的 AttributeError 异常

KeyError: 'flavor'

## 处理无效属性名

FrozenJSON 有个缺陷，没有对名称为 Python 关键字的属性做特殊处理。比如说像下面这样构建一个对象：

```
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})
```

此时无法读取 grad.class 的值，因为 Python 中 class 是保留关键字

```
  File "<ipython-input-29-742ecb9642c1>", line 2
    grad.class
             ^
SyntaxError: invalid syntax
```

当然，也可以这么做：

```
getattr(grad, 'class')
# 1982
```
 
但是， FrozenJSON 类的目的是为了便于访问数据，因此更好的方法是检查传给 `FrozenJSON.__init__` 方法的映射中是否有关键字名称，如果有在键名后加上 `_`，然后通过下面方式读取

```
grad.class
```

In [48]:
from collections import abc
import keyword

class FrozenJSON:
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): # 判断是不是保留字
                key += '_'
            self.__data[key] = value
    def __getattr__(self, name): 
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name]) 
    
    @classmethod 
    def build(cls, obj): 
        if isinstance(obj, abc.Mapping): 
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [50]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})
grad.class_

SyntaxError: invalid syntax (<ipython-input-50-1081492d1dd5>, line 3)

如果 JSON 对象中的键不是有效 Python 标识符，也会遇到类似问题

```
grad.2be

  File "<ipython-input-50-1081492d1dd5>", line 3
    grad.2be
         ^
SyntaxError: invalid syntax
```

这个问题在 Python 3 中很好解决，可以用 str 提供的 s.isidentifier() 方法能根据语言的语法判断 s 是否为有效的 Python 标识符。但是，把无效的标识符变成有效的属性名不容易，对此，有两个很简单的解决办法，一个是抛出异常，另一个是把无效的键转成通用名称，例如 `attr_0`，`attr_1` 等等。这里不做演示

## 使用 `__new__` 方法以灵活的方式创建对象

我们通常把 `__init__` 称为构造方法，这是从其他语言借鉴的术语，其实用于构建示例的特殊方法是 `__new__`，这是个类方法（使用特殊方式处理，因此不用加 @classmethod 装饰器），必须返回一个实例。返回的实例会作为第一个参数（即 self）传给 `__init__` 方法。因为调用 `__init__` 方法时要传入实例，而且禁止返回任何值，所以 `__init__` 其实就是初始化方法。真正的构造方法是 `__new__`。我们几乎不需要自己编写 `__new__` 方法，因为从 object 类继承的实现就够用了

刚才说明的过程是 `__new__` 方法到 `__init__` 方法，是最常见的，但不是唯一的。`__new__` 方法也可以返回其他类的实例，此时解释器不会调用 `__init__` 方法

也就是说。Python 构建对象的过程可以使用下面伪代码概括:

In [2]:
def object_maker(the_class, some_arg):
    new_object = the_class.__new__(some_arg)
    if isinstance(new_object, the_class):
        the_class.__init__(new_object, some_arg)
    return new_object

# 下面两个语句作用基本相同
# x = Foo('bar')
# x = object_maker(Foo, 'bar')

使用 `__new__` 代替 build 方法：

In [5]:
from collections import abc
import keyword

class FrozenJSON:
    
    def __new__(cls, arg): # 这是类方法，第一个参数是类本身，余下的参数与 `__init__` 方法一样，只不过没有 self
        if isinstance(arg, abc.Mapping):
            # 默认行为是委托给超类的 __new__` 方法。这里调用的是 object 基类的 __new__ 方法，把唯一的参数设为 FrozenJSON
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): # 判断是不是保留字
                key += '_'
            self.__data[key] = value
    def __getattr__(self, name): 
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])  # 之前调用的是 build 方法，现在直接调用构造方法